# <center> Water Jug Problem </center>

#### Mohammad Soban Shaikh  &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 21BCP296

In [7]:
import numpy as np

In [8]:
class Node:
    def __init__(self, state, parent, action):
        self.state = state
        self.parent = parent
        self.action = action

In [9]:
class StackFrontier:
    def __init__(self):
        self.frontier = []

    def add(self, node):
        self.frontier.append(node)

    def contains_state(self, state):
        return any((node.state == state).all() for node in self.frontier)

    def empty(self):
        return len(self.frontier) == 0

    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[-1]
            self.frontier = self.frontier[:-1]
            return node

In [10]:
class QueueFrontier(StackFrontier):
    def remove(self):
        if self.empty():
            raise Exception("Empty Frontier")
        else:
            node = self.frontier[0]
            self.frontier = self.frontier[1:]
            return node

In [21]:
# Solve the water jug problem with bfs and dfs
class WaterJugProblem:
    def __init__(self, capacity, goal):
        # Capacity is a tuple of the capacities of the two jugs
        self.capacity = capacity 
        self.goal = goal
        self.initial_state = np.array([0,0])

    
    def actions(self, state):
        actions = ["Fill Jug 1", "Fill Jug 2", "Empty Jug 1", "Empty Jug 2", "Pour Jug 1 to Jug 2", "Pour Jug 2 to Jug 1"]
        return actions

    
    def result(self, state, action):
        # If action is to fill jug 1, then self.capacity[0] is the maximum amount of water that can be filled in jug 1
        if action == "Fill Jug 1":
            return np.array([self.capacity[0], state[1]])
        
        # If the action is to fill jug 2, then self.capacity[1] is the maximum amount of water that can be filled in jug 2
        elif action == "Fill Jug 2":
            return np.array([state[0], self.capacity[1]])
        
        # If the action is to empty jug 1, then 0 is the amount of water in jug 1
        elif action == "Empty Jug 1":
            return np.array([0, state[1]])
        
        # If the action is to empty jug 2, then 0 is the amount of water in jug 2
        elif action == "Empty Jug 2":
            return np.array([state[0], 0])
        
        # If the action is to pour water from jug 1 to jug 2, then the amount of water that can be poured is the minimum of the amount of water in jug 1 and the amount of space left in jug 2
        elif action == "Pour Jug 1 to Jug 2":
            pour = min(state[0], self.capacity[1] - state[1])
            return np.array([state[0] - pour, state[1] + pour])
        
        elif action == "Pour Jug 2 to Jug 1":
            pour = min(self.capacity[0] - state[0], state[1])
            return np.array([state[0] + pour, state[1] - pour])
        
        else:
            raise Exception("Invalid action")
        
    
    def goal_test(self, state):
        return np.array_equal(state, self.goal)


    def solve(self, strategy):
        start = Node(self.initial_state, None, None)
        frontier = strategy()
        frontier.add(start)
        
        explored = set()
        
        while True:
            if frontier.empty():
                raise Exception("No solution")
            node = frontier.remove()

            # Check if the current state is the goal state
            if self.goal_test(node.state):
                if strategy == StackFrontier:
                    print("The solution is found using DFS")

                elif strategy == QueueFrontier:
                    print("The solution is found using BFS")

                actions = []

                # while the current state is not the initial state, add the action that led to the current state to the actions list
                while node.parent is not None:
                    actions.append(node.action)
                    node = node.parent

                # Actions are stored as a list from the goal state to the initial state. Reverse the list to get the actions from the initial state to the goal state
                actions.reverse()
                return actions
            
            # If the current state is not the goal state, then add the current state to the explored set and add all the child states to the frontier
            explored.add(tuple(node.state))

            if strategy == StackFrontier:
                for action in reversed(self.actions(node.state)):
                    child_state = self.result(node.state, action)
                    if not frontier.contains_state(child_state) and tuple(child_state) not in explored:
                        child = Node(child_state, node, action)
                        frontier.add(child)


            elif strategy == QueueFrontier:
                for action in self.actions(node.state):
                    child_state = self.result(node.state, action)
                    if not frontier.contains_state(child_state) and tuple(child_state) not in explored:
                        child = Node(child_state, node, action)
                        frontier.add(child) 

            else:
                raise Exception("Invalid strategy")

In [23]:
if __name__ == "__main__":
    problem = WaterJugProblem(np.array([4,3]), np.array([2,0]))
    print(problem.solve(StackFrontier))

The solution is found using DFS
['Fill Jug 1', 'Pour Jug 1 to Jug 2', 'Empty Jug 2', 'Pour Jug 1 to Jug 2', 'Fill Jug 1', 'Pour Jug 1 to Jug 2', 'Empty Jug 2']
